In [1]:
import pandas as pd
import altair as alt
import numpy as np

In [51]:
# Color mapping for different modes
modes_linelle=['HiFi-only (alt)','HiFi-only (pri)','HiC hap1','HiC hap2','Trio hap1 (pat)','Trio hap2 (mat)']
colors_linelle=['#a6cee3','#1f78b4','#b2df8a', '#33a02c', '#fb9a99','#e31a1c']

modes_byung=['HiFi-only','HiFi-Hic','HiFi-trio','CLR']
colors_byung=['#1f78b4', '#33a02c', '#e31a1c','#ff7f00']

## Linelle's graphs

In [11]:
def hapmer_graph(df):
    graph = alt.Chart(df).mark_point(clip=True).encode(
        x = alt.X(
            'pat_hapmer:Q',
            scale=alt.Scale(
                type='log',
                domain=(10,2000000)
            ),
            axis=alt.Axis(
                values=[10,100,1000,10000,1000000]
            ),
            title="Paternal hapmers"
        ),
        y = alt.Y(
            'mat_hapmer:Q',
            scale=alt.Scale(
                type='log',
                domain=(10,2000000)
            ),
            axis=alt.Axis(
                values=[10,100,1000,10000,1000000]
            ),
            title="Maternal hapmers"
        ),
        color=alt.Color(
                    'Assembly',
                    scale=alt.Scale(
                    domain=modes_linelle,
                range=colors_linelle
                    )
                ),    
        size=alt.Size('Size:Q',scale=alt.Scale(range=[100, 2000],domain=(10,20000000)))
    ).configure_axis(
        labelFont='Arial',
        labelFontSize = 30,
        titleFont='Arial',
        titleFontSize=30
    ).configure_legend(
        orient='top-left',
        fillColor='white',
        labelFont='Arial',
        labelFontSize=23,
        title=None,
        symbolSize=200,
        symbolStrokeWidth=5,
    ).properties(
        height=800,
        width=800
    )
    return(graph)

In [4]:
blob = pd.read_csv('hicsolotrio.allcounts.10x.count',sep='\t')

In [5]:
blob.head()

,Assembly,Contig,pat_hapmer,mat_hapmer,Size
0,bTaeGut2.trim.HiC.hic.hap1.p_ctg,h1tg000001l,5,1012249,11321021
1,bTaeGut2.trim.HiC.hic.hap1.p_ctg,h1tg000002l,467388,203,14597216
2,bTaeGut2.trim.HiC.hic.hap1.p_ctg,h1tg000003l,20,654014,4314514
3,bTaeGut2.trim.HiC.hic.hap1.p_ctg,h1tg000004l,755214,172,5769299
4,bTaeGut2.trim.HiC.hic.hap1.p_ctg,h1tg000005l,849658,872,18748363


In [6]:
name_map = {}
name_map['bTaeGut2.solo.p2']='HiFi-only (alt)'
name_map['bTaeGut2.solo.p1']='HiFi-only (pri)'
name_map['bTaeGut2.trim.HiC.hic.hap1.p_ctg']='HiC hap1'
name_map['bTaeGut2.trim.HiC.hic.hap2.p_ctg']='HiC hap2'
name_map['bTaeGut2.trio.cutadapt.20211115.dip.hap1.p_ctg']='Trio hap1 (pat)'
name_map['bTaeGut2.trio.cutadapt.20211115.dip.hap2.p_ctg']='Trio hap2 (mat)'

In [7]:
for key in name_map.keys():
    blob.loc[ (blob['Assembly'] == key,'Assembly') ] =  name_map[key]

In [8]:
panel_a = hapmer_graph(blob)
panel_a

alt.Chart(...)

In [9]:
blob = blob[blob['pat_hapmer'] > 50]
blob = blob[blob['mat_hapmer'] > 50]
blob = blob[blob['mat_hapmer']/(blob['mat_hapmer']+blob['pat_hapmer'])<=.95]
blob = blob[blob['pat_hapmer']/(blob['mat_hapmer']+blob['pat_hapmer'])<=.95]

In [12]:
panel_a = hapmer_graph(blob)
panel_a

alt.Chart(...)

In [13]:
all_2copy = pd.read_csv('all_2copy.tsv',sep='\t')

In [14]:
all_2copy.head()

,Copies,kmer_multiplicity,Count,Assembly,ordering
1,2,20,19707,Trio hap1 (pat),1
2,2,21,20797,Trio hap1 (pat),1
3,2,22,21937,Trio hap1 (pat),1
4,2,23,22992,Trio hap1 (pat),1
5,2,24,24552,Trio hap1 (pat),1


In [15]:
def line_plot(df,cats,cols):
    return(
        alt.Chart(df).mark_line(clip=True,strokeWidth=20,opacity=.75).encode(
            y = alt.Y(
                'Count:Q',
                scale=alt.Scale(domain=(0,2000000)),
                axis=alt.Axis(values=[0,500000,1000000,1500000,2000000]),
                title='Count'
            ),
            x = alt.X(
                'kmer_multiplicity:Q',
                scale=alt.Scale(domain=(49,91)),
                axis=alt.Axis(values=[50,60,70,80,90]),
                title='k-mer multiplicity'
            ),
            color=alt.Color(
                'Assembly',
                scale=alt.Scale(
                    #scheme='paired',
                    domain=cats,
                    range=cols
                )
            )
        ).properties(
            height=800,
            width=800
        )
    )

In [52]:
panel_b = line_plot(all_2copy,modes_linelle,colors_linelle)
panel_b

alt.Chart(...)

In [17]:
all_readonly = pd.read_csv('all_readonly.tsv',sep='\t')

In [18]:
all_readonly.head()

,Copies,kmer_multiplicity,Count,Assembly,ordering
1,read-only,20,4929927,HiC hap1,NaN
2,read-only,21,5330840,HiC hap1,NaN
3,read-only,22,5753258,HiC hap1,NaN
4,read-only,23,6179953,HiC hap1,NaN
5,read-only,24,6603988,HiC hap1,NaN


In [53]:
panel_c = line_plot(all_readonly,modes_linelle,colors_linelle)
panel_c

alt.Chart(...)

In [22]:
alt.hconcat(panel_b,panel_c).configure_axis(
            labelFont='Arial',
            labelFontSize = 30,
            titleFont='Arial',
            titleFontSize=30
        ).configure_legend(
            orient='left',
            fillColor='white',
            labelFont='Arial',
            labelFontSize=23,
            title=None,
            symbolSize=1000,
            symbolStrokeWidth=20,
        )

alt.HConcatChart(...)

## ByungJuneKo figures

In [23]:
dup = pd.read_csv('Kmer_Dupl.txt',sep='\t',names=['mode','prct'])

In [24]:
dup

,mode,prct
0,CLR,1.872370
1,HiFi-only,1.596290
2,HiFi-Hic,0.892503
3,HiFi-trio,0.726224


In [25]:
panel_B_a = alt.Chart(dup).mark_bar(color='green',opacity=.7).encode(
    x=alt.X('mode',title=None),
    y=alt.Y('prct',title="Proportion of k-mers (%)"),
    color=alt.Color(
                'mode',
                scale=alt.Scale(
                    domain=modes_byung,
                    range=colors_byung
                ),
            legend=None
        
            ),    
    
).properties(
    width=800,
    height=800,
)
panel_B_a

alt.Chart(...)

In [26]:
exp_col = pd.read_csv('Kmer_ExpCollap.txt',sep='\t',names=['mode','prct','tp'])

In [27]:
exp_col

,mode,prct,tp
0,CLR,10.86030,Expansion
1,HiFi-only,9.40783,Expansion
2,HiFi-Hic,10.13960,Expansion
3,HiFi-trio,9.55756,Expansion
4,CLR,9.81460,Collapse
5,HiFi-only,6.53933,Collapse
6,HiFi-Hic,6.37083,Collapse
7,HiFi-trio,6.79628,Collapse


In [28]:
panel_B_b = alt.Chart(exp_col).mark_bar(opacity=.7).encode(
    column=alt.Column('mode:N',header=alt.Header(orient="bottom",labelFontSize=40),title=None),
    x=alt.X('tp:N',title=None),
    y=alt.Y('prct',title="Proportion of k-mers (%)"),
    color=alt.Color(
                'mode',
                scale=alt.Scale(
                    domain=modes_byung,
                    range=colors_byung,
                ),
        legend=None
            ),
    opacity=alt.Opacity('tp:N',title=None,legend=None),
).properties(
    width=800,
    height=800,
).properties(width=200)

panel_B_b

alt.Chart(...)

In [29]:
pat = pd.read_csv('Paternal.txt',sep='\t',names=['tp','k_dup','k_col'])
mat = pd.read_csv('Maternal.txt',sep='\t',names=['tp','k_dup','k_col'])

In [30]:
pat

,tp,k_dup,k_col
0,Default,0.87,1.05910
1,Rebinned,0.83,1.05827


In [31]:
mat

,tp,k_dup,k_col
0,Default,4.37,1.06462
1,Rebinned,3.71,1.06444


In [32]:
def pat_mat(df,lower,upper,title):
    dup_line=alt.Chart(df).mark_line(
        color='red',
        opacity=.7,
        strokeWidth=20,
        point={
            "filled": True,
            "fill": "red",
            "size":800
        }
    ).encode(
        x=alt.X('tp',title=None),
        y=alt.Y('k_dup',scale=alt.Scale(domain=(lower, upper)),title="K-mer duplications (%)",axis=alt.Axis(titleColor='red')),
    )
    col_line=alt.Chart(df).mark_line(
        color='blue',
        opacity=.7,
        strokeWidth=20,
        point={
            "filled": True,
            "fill": "blue",
            "size":800
        }
    ).encode(
        x=alt.X('tp'),
        y=alt.Y(
            'k_col',
            title="K-mer collapse (%)",
            axis=alt.Axis(titleColor='blue')
            ),
    )
    graph = alt.layer(dup_line, col_line).resolve_scale(
        y = 'independent'
    ).properties(
        width=300,
        height=800,
        title=title
    )
    return(graph)

In [33]:
panel_B_c = (pat_mat(pat,0,5,"Male") | pat_mat(mat,0,5,"Female"))
panel_B_c

alt.HConcatChart(...)

In [34]:
panel_d = pd.read_csv('False_Duplication.txt',sep='\t',names=['mode','prct','mb'])

In [35]:
panel_d

,mode,prct,mb
0,CLR,1.29,13.8
1,HiFi-only,0.60,6.7
2,HiFi-Hic,0.21,2.4
3,HiFi-trio,0.12,1.3


In [36]:
bars = alt.Chart(panel_d).mark_bar(color='green',opacity=.7).encode(
    x=alt.X('mode',title=None),
    y=alt.Y('prct',title="Proportion of false duplications (%)"),
    color=alt.Color(
                'mode',
                legend=None,
                scale=alt.Scale(
                    domain=modes_byung,
                    range=colors_byung,
                ),
    ),
)
labels = alt.Chart(panel_d).mark_text(color='white',dy=45,fontSize=80
).encode(
    x=alt.X('mode',title="Mode"),
    y=alt.Y('prct'),
    text=alt.Text('mb'),
)

panel_B_d = (bars + labels).properties(
        width=800,
        height=800
    )
panel_B_d

alt.LayerChart(...)

In [37]:
panel_e_hm = pd.read_csv('False_Loss_HM.txt',sep='\t',names=['x','y','v1','v2','v3','v4','v5'])

In [38]:
panel_e_hm = pd.melt(panel_e_hm,id_vars=['x','y'],value_vars=['v3','v4','v5'])

In [39]:
panel_e_hm

,x,y,variable,value
0,CLR,CLR,v3,0.000000
1,HiFi-only,CLR,v3,0.074948
2,HiFi-Hic,CLR,v3,0.060714
3,HiFi-trio,CLR,v3,0.059711
4,CLR,HiFi-only,v3,0.019054
5,HiFi-only,HiFi-only,v3,0.000000
6,HiFi-Hic,HiFi-only,v3,0.021819
7,HiFi-trio,HiFi-only,v3,0.023188
8,CLR,HiFi-Hic,v3,0.013910
9,HiFi-only,HiFi-Hic,v3,0.031892


In [40]:
hm = alt.Chart(panel_e_hm).mark_rect().encode(
    x=alt.X('x:N',title=None),
    y=alt.Y('y:N',title=None),
    color=alt.Color('value:Q',scale=alt.Scale(scheme='greys'))
).properties(
    height=800,
    width=800
)


hm

alt.Chart(...)

In [41]:
panel_e_top = pd.read_csv('False_Loss.txt',sep='\t',names=['mode','frac','prct','mb'])

In [42]:
panel_e_top

,mode,frac,prct,mb
0,CLR,0.065124,6.512446,69.4
1,HiFi-only,0.021354,2.135355,24.1
2,HiFi-Hic,0.026575,2.657451,29.7
3,HiFi-trio,0.052841,5.284131,56.8


In [43]:
bars_e = alt.Chart(panel_e_top).mark_bar(color='green',opacity=.7).encode(
    x=alt.X('mode',title="Mode",axis=None),
    y=alt.Y('prct',title=["Proportion of","false losses (%)"]),
     color=alt.Color(
                'mode',
                legend=None,
                scale=alt.Scale(
                    domain=modes_byung,
                    range=colors_byung,
                ),
    ),
)

labels_e = alt.Chart(panel_e_top).mark_text(color='white',align="center", baseline="bottom",dy=60,fontSize=60
).encode(
    text='mb',
x=alt.X('mode',title="Mode",axis=None),
    y=alt.Y('prct')
)

top_hist = (bars_e + labels_e).properties(
    height=200,
    width=800,
)

In [44]:
top_hist

alt.LayerChart(...)

In [45]:
panel_B_e = (top_hist & hm)
panel_B_e

alt.VConcatChart(...)

In [46]:
alt.hconcat(panel_B_a,panel_B_b,panel_B_c).configure_axis(
            labelFont='Arial',
            labelFontSize = 30,
            titleFont='Arial',
            titleFontSize=30
        ).configure_title(
    fontSize=40,
    font='Arial',
)

alt.HConcatChart(...)

In [47]:
alt.hconcat(panel_B_d,panel_B_e).configure_axis(
            labelFont='Arial',
            labelFontSize = 30,
            titleFont='Arial',
            titleFontSize=30
        ).configure_legend(
        fillColor='white',
        labelFont='Arial',
        labelFontSize=40,
        title=None,
        symbolSize=200,
        symbolStrokeWidth=5,
    )

alt.HConcatChart(...)